## Data Setup
---


In [35]:
token = 'ea16075f94a8cd20ef4f8cac731b744d928b006a709fcc6322b83a20c26cfc5cfd0316ca9ad25cf65e663af971937ce7cd490dd7bd7ca50e88272206afa61f62'


In [6]:
import pandas

train = pandas.read_csv('../data/train.csv')

cnt_all = len(train)

survivors = train[train.Survived.eq(1)]
cnt_survivors = len(survivors)

prob_survivors = len(survivors) / cnt_all

round(prob_survivors, 2)

0.38

In [18]:
def get_modifier_pclass(pclass):
    cnt_surv_pclass = len(survivors[survivors.Pclass.eq(pclass)])

    p_cl_surv = cnt_surv_pclass / cnt_survivors

    p_cl = len(train[train.Pclass.eq(pclass)]) / cnt_all

    return p_cl_surv / p_cl

In [32]:
def get_modifier_sex(sex):
    cnt_surv_sex = len(survivors[survivors.Sex.eq(sex)])

    p_sex_surv = cnt_surv_sex / cnt_survivors

    p_sex = len(train[train.Sex.eq(sex)]) / cnt_all

    return p_sex_surv / p_sex

In [21]:
def pre_process(passenger):
    return [
        get_modifier_pclass(passenger["Pclass"]),
        get_modifier_sex(passenger["Sex"]),
    ]


In [33]:
print(pre_process(train.iloc[52]))

[1.6403508771929822, 1.9332048273550118]


In [34]:
print(pre_process(train.iloc[26]))

[0.6314181584306999, 0.49215543190732464]


## Quantum Code
---


In [38]:
from functools import reduce
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from math import asin, sqrt, ceil
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

In [41]:
def prob_to_angle(prob):
    """
    Converts given P(psi) value into an equivalent theta value.
    """

    return 2 * asin(sqrt(prob))

def pqc(backend, prior, modifiers, shots=1, hist=False, measure=False):

    qr = QuantumRegister(7)
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr, cr) if measure else QuantumCircuit(qr)

    trunks = 3
    aux = trunks + 1
    aux_half = trunks + 1
    aux_full = trunks + 2
    target = trunks + 3

    qc.ry(prob_to_angle(prior), target)

    qc.x(target)

    qc.cry(prob_to_angle(prior / (1 - prior)), target, aux_full)

    qc.cx(aux_full, target)

    qc.cry(prob_to_angle(0.5 * prior / (1 - (2 * prior))), target, aux_half)

    qc.x(target)
    qc.cx(aux_full, target)

    sorted_modifiers = sorted(modifiers)

    for step in range(0, len(modifiers)):
        if sorted_modifiers[step] > 1:
            qc.cry(prob_to_angle(min(1, sorted_modifiers[step] - 1)), aux_full, target)

            qc.ccx(target, aux_full, 0)
            qc.ccx(target, 0, aux_full)

            if step == 0:
                qc.cry(prob_to_angle(min(1, (sorted_modifiers[step] - 1) * 2 * 2)), aux_half, aux_full)
        else:
            qc.cry(prob_to_angle(1 - sorted_modifiers[step]), target, step * 2)
            qc.cx(step * 2, target)

    if measure:
        qc.measure(qr[0], cr[0])
        

    results = execute(qc, backend, shots=shots).result().get_counts()
    return plot_histogram(results, figsize=(12, 4)) if hist else results
    

In [43]:
def post_process(counts):
    return int(list(map(lambda item: item[0],
                        counts.items()))[0])
    

In [ ]:
def run(f_classify, data):
    return [f_classify(data.iloc[i]) for i in range(0, len(data))]
